In [31]:
using DynamicPolynomials
using TSSOS ## TODO: use the official TSSOS
using MAT
using LinearAlgebra
using Printf
using JuMP

In [32]:
@polyvar temp1[1:4, 1:4]
@polyvar temp2[1:4, 1:4]
temp_p = temp2 * 1.1 + temp1 * 1.1

function Joint2SE3(c, s, x, y, z)
    ## we always rotate about the z axis
    T = temp_p * 0
    T[1,1] = c
    T[2,2] = c
    T[1,2] = -s
    T[2,1] = s
    T[3,3] = 1
    
    T[1,4] = c * x - s * y
    T[2,4] = s * x + c * y
    T[3,4] = z
    T[4,4] = 1
    return T
end

function SE3_Euler_num(y, p, r, lx, ly, lz)
    T = temp_p * 0
    T[1:3, 1:3] = rotz_num(y) * roty_num(p) * rotx_num(r)
    T[4,4] = 1.0
    T[1,4] = lx
    T[2,4] = ly
    T[3,4] = lz
    return T
end

function rotx_num(t)
    R = temp_p[1:3, 1:3] * 0
    R[2,2] = cos(t)
    R[3,3] = cos(t)
    R[2,3] = -sin(t)
    R[3,2] = sin(t)
    R[1,1] = 1.0
    return R
end

function roty_num(t)
    R = temp_p[1:3, 1:3] * 0
    R[1,1] = cos(t)
    R[3,3] = cos(t)
    R[1,3] = sin(t)
    R[3,1] = -sin(t)
    R[2,2] = 1.0
    return R
end

function rotz_num(t)
    R = temp_p[1:3, 1:3] * 0
    R[1,1] = cos(t)
    R[2,2] = cos(t)
    R[1,2] = -sin(t)
    R[2,1] = sin(t)
    R[3,3] = 1.0
    return R
end

function quat2rot_num(w, x, y, z)
    R = temp_p[1:3, 1:3] * 0
    nq = sqrt(w^2 + x^2 + y^2 + z^2)
    w = w / nq
    x = x / nq
    y = y / nq
    z = z / nq
    
    R[1,1] = 1 - 2 * (y^2 + z^2)
    R[2,2] = 1 - 2 * (x^2 + z^2)
    R[3,3] = 1 - 2 * (x^2 + y^2)
    
    R[1,2] = 2 * (x * y - w * z)
    R[1,3] = 2 * (w * y + x * z)
    R[2,3] = 2 * (y * z - w * x)
    
    R[2,1] = 2 * (x * y + w * z)
    R[3,1] = 2 * (- w * y + x * z)
    R[3,2] = 2 * (y * z + w * x)
    return R
end

function cross_(x, y) 
    z = temp_p[1:3, 1]
    z[1] = -x[3] * y[2] + x[2] * y[3]
    z[2] =  x[3] * y[1] - x[1] * y[3]
    z[3] = -x[2] * y[1] + x[1] * y[2]
    return z
end

function add_SO3_cons(R, eq)
    T = R' * R
#     print(T[1,1] - 1.0)
    append!(eq, [T[1,1] - 1.0])
    append!(eq, [T[2,2] - 1.0])
    append!(eq, [T[3,3] - 1.0])
    append!(eq, [T[1,2]])
    append!(eq, [T[1,3]])
    append!(eq, [T[2,3]])
    
    append!(eq, cross_(R[1,:], R[2, :]) - R[3,:])
    append!(eq, cross_(R[2,:], R[3, :]) - R[1,:])
    append!(eq, cross_(R[3,:], R[1, :]) - R[2,:])
    
    return eq
end

function add_SE3_tran_cons(T1, T2, eq)
    T = T1 - T2
    
    append!(eq, [T[1, 1]])
    append!(eq, [T[1, 2]])
    append!(eq, [T[1, 3]])
    
    
    append!(eq, [T[2, 1]])
    append!(eq, [T[2, 2]])
    append!(eq, [T[2, 3]])
    
    
    append!(eq, [T[3, 1]])
    append!(eq, [T[3, 2]])
    append!(eq, [T[3, 3]])
    
    append!(eq, [T[1, 4]])
    append!(eq, [T[2, 4]])
    append!(eq, [T[3, 4]])    
    return eq
end

function add_SO3_tran_cons(T1, T2, eq)
    T = T1 - T2
    
    append!(eq, [T[1, 1]])
    append!(eq, [T[1, 2]])
    append!(eq, [T[1, 3]])
    
    
    append!(eq, [T[2, 1]])
    append!(eq, [T[2, 2]])
    append!(eq, [T[2, 3]])
    
    
    append!(eq, [T[3, 1]])
    append!(eq, [T[3, 2]])
    append!(eq, [T[3, 3]])
    
    return eq
end


function make_SE3(R, x, y, z)
    T = temp_p * 0
    T[1:3, 1:3] = R
    T[1,4] = x
    T[2,4] = y
    T[3,4] = z
    T[4,4] = 1.0
    return T
end

function SE3_inverse(T)
    Tnew = T
    Tnew[1:3, 1:3] = T[1:3, 1:3]'
    Tnew[1:3, 4] = - T[1:3, 1:3]' * T[1:3, 4]
    return Tnew
end

SE3_inverse (generic function with 1 method)

In [33]:
function batch_test(dof)
    ## generate random arm 
    x_bar = rand(dof, 1) #* 0.5
    y_bar = rand(dof, 1) #* 0
    z_bar = rand(dof, 1) #* 0
    
    ## generate random rotation offset
    T_list = zeros(dof, 4, 4)
    T_list[1, :, :] = Matrix(1.0I, 4, 4)
    T_list[:, 4, 4] .= 1.0

    for k = 1:dof
        T_list[k, 1:3, 1:3] = rotx_num(2 * rand(Float64) * pi) * roty_num(2 * rand(Float64) * pi) * rotz_num(2 * rand(Float64) * pi)
    end
    
    ## generate random feasible end pose by forward kinematics
    theta_list = randn(dof, 1)
    T_end = Matrix(1.0I, 4, 4)
    for k = 1:dof
        Act = Joint2SE3(cos(theta_list[k]), sin(theta_list[k]), x_bar[k], y_bar[k], z_bar[k])
        T_end = T_end * Act * T_list[k, :, :]
    end

    @polyvar R[1:dof+1, 1:3, 1:3] 
    @polyvar c[1:dof]
    @polyvar s[1:dof]
    @polyvar x[1:dof+1]
    @polyvar y[1:dof+1]
    @polyvar z[1:dof+1]
    var = append!(reshape(R, (1+dof) * 9), c, s, x, y, z)
    # print(var)
    
    ## base link
    eq = [temp_p[1,1] * 0]

    eq[1] = R[1, 1, 1] - 1.0
    append!(eq, [R[1, 1, 2] - 0.0])
    append!(eq, [R[1, 1, 3] - 0.0])
    append!(eq, [R[1, 2, 1] - 0.0])
    append!(eq, [R[1, 2, 2] - 1.0])
    append!(eq, [R[1, 2, 3] - 0.0])
    append!(eq, [R[1, 3, 1] - 0.0])
    append!(eq, [R[1, 3, 2] - 0.0])
    append!(eq, [R[1, 3, 3] - 1.0])

    append!(eq, [x[1] - 0.0])
    append!(eq, [y[1] - 0.0])
    append!(eq, [z[1] - 0.0])

    f_act = 0

    for k = 1:dof

        append!(eq, [c[k]^2 + s[k]^2 - 1]) # joint SO2 constrains

        Tk = make_SE3(R[k,:,:], x[k], y[k], z[k])
        Tkp1 = make_SE3(R[k+1,:,:], x[k+1], y[k+1], z[k+1])
        Act = Joint2SE3(c[k], s[k], x_bar[k], y_bar[k], z_bar[k])

        # eq = add_SE3_tran_cons(SE3_inverse(Tk) * Tkp1, T_list[k, :, :] * Act, eq)
        eq = add_SE3_tran_cons(Tkp1, Tk * Act * T_list[k, :, :], eq)
        eq = add_SO3_cons(R[k+1,:,:], eq) # SO3 constraints
        
        # ref angle: 0 for all links
        f_act = f_act + (1 - c[k])^2 + s[k]^2
    end
    
    ## end constraints 
    for i = 1:3
        for j = 1:3
            append!(eq, [ T_end[i,j] - R[end, i, j] ])
        end
    end

    append!(eq, [ x[end] - T_end[1, 4] ])
    append!(eq, [ y[end] - T_end[2, 4] ])
    append!(eq, [ z[end] - T_end[3, 4] ])


    ineq = []

    f = 1.0 * f_act #  + f_R + f_p # * 1e-5 # 1e-3 * sum(var.^2)
    pop = append!([f], ineq, eq)
    

    eq_len = length(eq)
    order = 2
    
    time_start = time()
    opt,sol,data = cs_tssos_first(pop, var, order, numeq=eq_len, CS="NC", TS=false, MomentOne=false, solution=true, QUIET=false);
    time_end = time()
    
    ## the sparse moment matrix
    moment = []
    for k = 1:length(data.moment)
        append!(moment, [convert(Matrix{Float64}, data.moment[k])]) # data.Mmatrix[k])
    end

    elapsed = time_end - time_start

    log = Dict("sol" => sol, "dof" => dof, "moment" => moment, "elapsed" => elapsed, "Tg" => T_end,
        "T" => T_list, "Th" => theta_list, "xb"=>x_bar, "yb"=>x_bar, "zb"=>x_bar, "flag"=>data.flag)
    return log
end

batch_test (generic function with 1 method)

In [34]:
for dof = [6] # , 7, 10, 15, 20, 25, 30, 40]
    for id = 1:1 # 50
        log = batch_test(dof)
        file_name = "log_" * string(dof) * "_" * string(id) * ".mat"
        matwrite(file_name, log)
    end
end

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
-----------------------------------------------------------------------------
The clique sizes of varibles:
[11, 9, 7, 8]
[11, 15, 18, 3]
-----------------------------------------------------------------------------
Obtained the variable cliques in 0.0017297 seconds. The maximal size of cliques is 11.
Assembling the SDP...
There are 21529 affine constraints.
SDP assembling time: 0.1020982 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 21529           
  Cones                  : 0               
  Scalar variables       : 21007           
  Matrix variables       : 47              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependen